# Question 5, Option C, Step I: Search for Nearest K Origination Point

Author's Workday ID: C175799, Initials: RPR

## Imports and Util Functions

In [1]:
import pandas as pd
import numpy as np
import time
import datetime
from math import radians, cos, sin, asin, sqrt
from haversine import haversine
import math
import random
from multiprocessing import Process, current_process
import hashlib

def get_time( output=True ):
    
    """
    Simple util to get system time and dump it to console automagically
    """
    temp = time.time()
    if output:
        now = datetime.datetime.now()
        print( now.strftime( "%Y.%m.%d %H:%M" ) )
        
    return temp

foo = get_time()

def print_time( start_time, end_time, interval="seconds" ):
    
    """
    Simple util to calculate elapsed time and dump it to console automagically
    """
    
    if interval == "hours":
        print ( "Time to process: [%s] hours" % ( str( ( end_time - start_time ) / 60 / 60 ) ) )
    else:
        print ( "Time to process: [%s] seconds" % ( str( end_time - start_time ) ) )

print_time( 0, 3600 )
print_time( 0, 3600, interval="hours" )

2018.05.09 13:27
Time to process: [3600] seconds
Time to process: [1.0] hours


## Load Trips Data, Previously Cleaned for Q4

In [2]:
start_time = get_time()
trips = pd.read_csv( "data/green-tripdata-2015-09-cleaned.csv" )
end_time = get_time()
print_time( start_time, end_time )

# drop superfluous fields
trips.drop( columns=[ "vendorid", "store_and_fwd_flag", "ratecodeid", "passenger_count", "fare_amount", "extra", "mta_tax", "tip_amount", "tolls_amount", "improvement_surcharge", "total_amount", "payment_type", "trip_type", "tip_percent", "credit_card", "tip_recorded" ], inplace=True )

2018.05.09 13:27
2018.05.09 13:27
Time to process: [2.64208722114563] seconds


## Create Lookup Table and Function to Calculate Mean MPH 

In [3]:
# build lookup table for mph by day and hour, takes advantage of Panda's indexing of group_by indexing.
trips_mph_day_n_hour_grp = trips.groupby( [ "pickup_day_of_week", "pickup_hour" ] )
trips_mph_day_n_hour = trips_mph_day_n_hour_grp[ "speed_mph" ].mean()

def get_mph( day, hour ):
    
    return trips_mph_day_n_hour[ day ][ hour ]

now = datetime.datetime.now()
get_mph( now.weekday(), now.hour )

11.249633747142711

## Calculate Time to Traverse City from Where Taxis is to Where Pickup Is

In [4]:
def get_time_to_pickup( pickup_day, pickup_hour, pickup_lat_lon, taxi_lat_lon, interval="minutes", debug="False" ):
    
    distance = haversine( pickup_lat_lon, taxi_lat_lon, miles=True )
    mph = get_mph( pickup_day, pickup_hour )
    
    if debug:
        print( "distance:", distance, "miles @", mph, "mph" )
        
    if ( interval == "hours" ):
        return ( distance / mph )
    else: # defaults to minutes
        return ( distance / mph ) * 60

### Test get_time_to_pickup(...)

In [5]:
# build time from here to there at a given time of day
from_lat = 40.7128
from_lon = -73.979485
#to_lat = 40.6128
#to_lon = -73.879485

pickup_location = ( from_lat, from_lon )
taxi_location = ( from_lat - 0.01, from_lon - 0.01 )

print( "Minutes to pickup:", get_time_to_pickup( now.weekday(), now.hour, pickup_location, taxi_location, debug=True ) )
print()
print( "  Hours to pickup:", get_time_to_pickup( now.weekday(), now.hour, pickup_location, taxi_location, interval="hours", debug=True ) )


distance: 0.8670130524375171 miles @ 11.2496337471 mph
Minutes to pickup: 4.62422015823

distance: 0.8670130524375171 miles @ 11.2496337471 mph
  Hours to pickup: 0.0770703359705


### What Are the Means?

In [6]:
# what's the mean mph across all trips?
mean_mph = trips.speed_mph.mean()
print( "Mean MPH:", mean_mph )
mean_distance = trips.trip_distance.mean()
print( "Mean Distance:", mean_distance )
mean_duration = trips.trip_duration_mins.mean()
print( "Mean Duration:", mean_duration, "minutes" )

Mean MPH: 12.477215717
Mean Distance: 2.85825795027
Mean Duration: 20.4071485747 minutes


## Calculate Bearing from Pickup to Dropoff: Moved to Data Cleanup

In [7]:
# # From: https://gist.github.com/jeromer/2005586
# def calculate_initial_compass_bearing( pointA, pointB ):
#     """
#     Calculates the bearing between two points.
#     The formulae used is the following:
#         θ = atan2(sin(Δlong).cos(lat2),
#                   cos(lat1).sin(lat2) − sin(lat1).cos(lat2).cos(Δlong))
#     :Parameters:
#       - `pointA: The tuple representing the latitude/longitude for the
#         first point. Latitude and longitude must be in decimal degrees
#       - `pointB: The tuple representing the latitude/longitude for the
#         second point. Latitude and longitude must be in decimal degrees
#     :Returns:
#       The bearing in degrees
#     :Returns Type:
#       float
#     """
#     if (type(pointA) != tuple) or (type(pointB) != tuple):
#         raise TypeError("Only tuples are supported as arguments")

#     lat1 = math.radians(pointA[0])
#     lat2 = math.radians(pointB[0])

#     diffLong = math.radians(pointB[1] - pointA[1])

#     x = math.sin(diffLong) * math.cos(lat2)
#     y = math.cos(lat1) * math.sin(lat2) - (math.sin(lat1)
#             * math.cos(lat2) * math.cos(diffLong))

#     initial_bearing = math.atan2(x, y)

#     # Now we have the initial bearing but math.atan2 return values
#     # from -180° to + 180° which is not what we want for a compass bearing
#     # The solution is to normalize the initial bearing as shown below
#     initial_bearing = math.degrees(initial_bearing)
#     compass_bearing = (initial_bearing + 360) % 360

#     return compass_bearing

### Test calculate_initial_compass_bearing(...)

In [1]:
# # get bearing between dc and nyc: https://www.distancefromto.net/distance-from-washington-d-c-to-new-york
# dc = ( 38.90719, -77.03687 )
# nyc = ( 40.71278, -74.00594 )
# print( calculate_initial_compass_bearing( dc, nyc ) )
# print( calculate_initial_compass_bearing( nyc, dc ) )

## Create Wrapper for Calculating Bearing from Pickup to Dropoff Lat/Lons

In [9]:
# def get_bearing( row ):
    
#     pickup = ( row.pickup_latitude, row.pickup_longitude )
#     dropoff = ( row.dropoff_latitude, row.dropoff_longitude )
    
#     return calculate_initial_compass_bearing( pickup, dropoff )

## Calculate Bearing for All Pickups in Trips DF

In [2]:
# start_time = get_time()
# trips[ "pickup_bearing" ] = trips.apply( get_bearing, axis=1 )
# print_time( start_time, get_time()  )

## Create lat/lon bins for Search and Simulation Optimization

In [11]:
# build pre-calc'd lookup for n-closest taxis
trips[ "pickup_latitude_bin_1" ] = round( trips.pickup_latitude, 1 )
trips[ "pickup_latitude_bin_2" ] = round( trips.pickup_latitude, 2 )
trips[ "pickup_longitude_bin_1" ] = round( trips.pickup_longitude, 1 )
trips[ "pickup_longitude_bin_2" ] = round( trips.pickup_longitude, 2 )


## Find Nearby Taxis w/in a Given Lat/Lon Window, for a Given Day/Hour

In [12]:
def get_taxis_by_pickups( day, hour, lat_full, lon_full, rounding_places, margin ):

    """
    Queries trips df using full precision pickup lat/long.
    Uses a combination of rounding of taxi hailer's lat/lons + margin value to create a window for nearby taxis.
    Because we're querying df of a month's worth of pickups, we also need to use 'day_of_week_mean' as shorthand 
    for 'on average, how many taxis are picking up on a given day and hour w/in this lat/lon window.
    """
    # To calculate mean availability per hour bin, we need to create a simple heuristic: 
    # i.e., how many mondays do we have in this month?
    day_of_week_mean = 30 / 7 # 4.285714285714286
    # TODO: This is a quick/gross approximation.  Programmatically get an actual count day of week for *this* month.
    
    lat = round( lat_full, rounding_places )
    lon = round( lon_full, rounding_places )
    
    # create subqueries
    trips_by_day  = trips.pickup_day_of_week == day
    trips_by_hour = trips.pickup_hour == hour
    lat_min = lat - margin
    lat_max = lat + margin
    lon_min = lon - margin
    lon_max = lon + margin
    trips_lat_min = trips.pickup_latitude_bin_2 >= lat_min
    trips_lat_max = trips.pickup_latitude_bin_2 <= lat_max
    trips_lon_min = trips.pickup_longitude_bin_2 >= lon_min
    trips_lon_max = trips.pickup_longitude_bin_2 <= lon_max
    
    # Intersection of subqueries
    candidates = trips[ ( ( trips_by_day ) & ( trips_by_hour ) & ( trips_lat_min ) & ( trips_lat_max ) & ( trips_lon_min ) & ( trips_lon_min ) ) ].copy()
    
    # stash these values for port-simulation analysis
    candidates[ "callers_lat" ] = lat_full
    candidates[ "callers_lon" ] = lon_full
    candidates[ "callers_day" ] = day
    candidates[ "callers_hour" ] = hour
    
    # calculate how many pickups per hour for this day of week, given that we're looking at a month's worth
    # of day(s), i.e., how many Tuesdays are there in this month?  30 / 7
    candidates_count = len( candidates )
    # this gives us mean pickups per hour in this candidate set...
    candidates[ "pickups_per_hour" ] = candidates_count / day_of_week_mean
    #...and thus, a mean wait time: 60 mins per hour / how many mean pickups occur in this hour of day?
    candidates[ "pickup_mean_wait_time_mins" ] = 60 / candidates.pickups_per_hour 
    
    return candidates


### Quick Test of get_taxis_by_pickups(...)

In [13]:
day = 0
hour = 0
lat_full = 40.5773838891
lon_full = -73.9788593716

# lat = round( lat_full, 2 )
# lon = round( lon_full, 2 )

in_this_quad = get_taxis_by_pickups( day, hour, lat_full, lon_full, 2, 0.01 )
in_this_quad

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_distance,trip_duration_mins,pickup_day_of_week,pickup_day,pickup_hour,pickup_minute,...,pickup_latitude_bin_1,pickup_latitude_bin_2,pickup_longitude_bin_1,pickup_longitude_bin_2,callers_lat,callers_lon,callers_day,callers_hour,pickups_per_hour,pickup_mean_wait_time_mins
290928,-73.980988,40.575558,-73.978432,40.575394,0.16,0.883333,0,7,0,4,...,40.6,40.58,-74.0,-73.98,40.577384,-73.978859,0,0,3.966667,15.12605
291384,-73.932617,40.584049,-73.905602,40.623772,5.04,12.450000,0,7,0,15,...,40.6,40.58,-73.9,-73.93,40.577384,-73.978859,0,0,3.966667,15.12605
291650,-73.981422,40.577431,-74.033134,40.625690,7.64,17.650000,0,7,0,18,...,40.6,40.58,-74.0,-73.98,40.577384,-73.978859,0,0,3.966667,15.12605
291761,-73.981224,40.575764,-74.026817,40.620869,4.93,11.333333,0,7,0,26,...,40.6,40.58,-74.0,-73.98,40.577384,-73.978859,0,0,3.966667,15.12605
292662,-73.981819,40.577023,-73.952812,40.695019,16.34,34.650000,0,7,0,29,...,40.6,40.58,-74.0,-73.98,40.577384,-73.978859,0,0,3.966667,15.12605
292913,-73.978577,40.575581,-73.981323,40.576786,3.24,18.233333,0,7,0,49,...,40.6,40.58,-74.0,-73.98,40.577384,-73.978859,0,0,3.966667,15.12605
293103,-73.981499,40.576031,-73.977539,40.606960,2.80,13.583333,0,7,0,39,...,40.6,40.58,-74.0,-73.98,40.577384,-73.978859,0,0,3.966667,15.12605
293182,-73.943581,40.583920,-73.953896,40.639648,3.90,12.283333,0,7,0,33,...,40.6,40.58,-73.9,-73.94,40.577384,-73.978859,0,0,3.966667,15.12605
646639,-73.981712,40.576748,-73.985504,40.598392,1.51,5.583333,0,14,0,30,...,40.6,40.58,-74.0,-73.98,40.577384,-73.978859,0,0,3.966667,15.12605
646673,-73.981491,40.576538,-74.023529,40.634754,7.30,15.766667,0,14,0,37,...,40.6,40.58,-74.0,-73.98,40.577384,-73.978859,0,0,3.966667,15.12605


In [14]:
def get_wait_time( row ):
    
    from_point = ( row.callers_lat, row.callers_lon )
    to_point = ( row.pickup_latitude, row.pickup_longitude )
    
    return get_time_to_pickup( row.callers_day, row.callers_hour, from_point, to_point, debug=False )    

## Help Function: Get Delta Between Two Bearings

In [31]:
# https://stackoverflow.com/questions/45659723/calculate-the-difference-between-two-compass-headings-python
def get_bearing_delta( init, final ):

    diff = final - init
    abs_diff = abs( diff )

    delta = 0
    if abs_diff == 180:
        delta = abs_diff
    elif abs_diff < 180:
        delta = diff
    elif final > init:
        delta = abs_diff - 360
    else:
        delta = 360 - abs_diff

    return delta

print("init -- final -- diff")
print("360 -- 1 -- {}".format(get_bearing_delta(360, 1)))
print("50 -- 60 -- {}".format(get_bearing_delta(50, 60)))
print("1 -- 360 -- {}".format(get_bearing_delta(1, 360)))


init -- final -- diff
360 -- 1 -- 1
50 -- 60 -- 10
1 -- 360 -- -1


In [32]:
# wrapper for bearings delta above
def get_bearing_delta_wrapper( row ):
    
    me = row.my_bearing
    taxi = row.pickup_bearing 
    
    # return the abs value, don't care about directionality
    return abs( get_bearing_delta( me, taxi ) )
    

## Build Get Candidates Near Me Function

In [33]:
def get_candidate_list( day, hour, my_lat, my_lon, my_bearing, rounding_places=2, search_margin=0.03, max_delta=90, max_wait_mins=10 ):

    """
    Takes the preliminary results from 'get_taxis_by_pickups(...)' and further filters it by bearing and wait times
    TODO: Merge this and 'get_taxis_by_pickups(...)' into one function.
    """
    
    taxis_near_me = get_taxis_by_pickups( day, hour, my_lat, my_lon, rounding_places, search_margin )

    if len( taxis_near_me ) == 0:
        return None
    
    taxis_near_me[ "pickup_time_between_pickups_mins" ] = taxis_near_me.apply( get_wait_time, axis=1 )
    taxis_near_me[ "pickup_total_wait_mins" ] = taxis_near_me.pickup_mean_wait_time_mins + taxis_near_me.pickup_time_between_pickups_mins
    taxis_near_me.sort_values( by="pickup_total_wait_mins", inplace=True )

    quad_summary = taxis_near_me[ [ "pickup_mean_wait_time_mins", "pickup_time_between_pickups_mins", "pickup_total_wait_mins", "pickup_bearing" ] ].copy()
    quad_summary[ "day" ] = day
    quad_summary[ "hour" ] = hour
    quad_summary[ "my_bearing" ] = my_bearing
    
    # D'oh! This doesn't work, simpleton! :-(
    #quad_summary[ "delta" ] = abs( my_bearing - quad_summary.pickup_bearing )
    quad_summary[ "delta" ] = quad_summary.apply( get_bearing_delta_wrapper, axis=1 )
        
    quad_summary[ "my_lat" ] = my_lat
    quad_summary[ "my_lon" ] = my_lon
    quad_summary[ "hash" ] = hashlib.md5( ( str( my_lat ) + str( my_lon ) + str( my_bearing ) ).encode() ).hexdigest()
    
    quad_summary.columns = [ "day", "hour", "mean_wait_mins", "distance_mins", "total_mins", "taxi_bearing", "my_bearing", "delta", "my_lat", "my_lon", "hash" ]

    quad_summary_finalists = quad_summary[ ( quad_summary.delta <= max_delta ) & ( quad_summary.total_mins <= max_wait_mins ) ].copy()
    quad_summary_finalists.sort_values( by="delta", inplace=True )
    
    return quad_summary_finalists

## Build Pickup Bins List for Simulation

In [34]:
pickup_points = trips[ [ "pickup_latitude_bin_2", "pickup_longitude_bin_2" ] ].copy()
pickup_points.sort_values( by=[ "pickup_latitude_bin_2", "pickup_longitude_bin_2" ], inplace=True )
print( len( pickup_points ) )
# we don't want ~1.5M pickup points, we want...
pickup_points_unique = pickup_points.drop_duplicates()
pickup_points_list = list( pickup_points_unique.itertuples( index=False, name=None ) )
#... 998 of them?
print( len( pickup_points_list ) )

# # grab a 10% sample to check out parallel processing
# # subsample: https://stackoverflow.com/questions/6482889/get-random-sample-from-list-while-maintaining-ordering-of-items
# sample_size = int( len( pickup_points_list ) * 0.1 )
# pickup_points_list = [ pickup_points_list[i] for i in sorted( random.sample( range( len( pickup_points_list ) ), sample_size ) ) ]
# print( len( pickup_points_list ) )

print( pickup_points_list[ 0:4 ] )
# randomize so that one core doesn't get the densest lat/lon bin
random.shuffle( pickup_points_list )
print( pickup_points_list[ 0:4 ] )

# Now useless speculation!
# seconds_per_pair = 0.2 # previous performance
# hours_to_process_uniques = len( pickup_points_list ) * seconds_per_pair * 7 * 24 / 60 / 60 / 7
# print( "     unique lat/lon coordinates", pickup_points_unique.shape[ 0 ] )
# print( "    sampled lat/lon coordinates", len( pickup_points_list ) )
# print( "  hours_to_process_uniques FAST", hours_to_process_uniques )
# print( "  hours_to_process_uniques SLOW", hours_to_process_uniques * 7 )

1455831
988
[(39.36, -74.44), (40.17, -74.18), (40.45, -74.29), (40.5, -74.45)]
[(40.76, -73.7), (40.89, -73.63), (40.85, -73.98), (40.73, -73.6)]


## Create Pickup Bin Chunks for Parallel Processing

Chunks = The number of cores available for processing

In [35]:
###################################################################################################
# I've got 12 available, but I want to leave 2 free for other things while I wait
###################################################################################################
chunks = 10

records = len( pickup_points_list )
chunk_size = int( records / chunks )
print( "records", records )
print( "chunk_size", chunk_size )

chunks_list = []

for ith in range( 0, chunks ):
    
    start = ith * chunk_size
    
    if ith == chunks - 1:
        end = records
    else:
        end = start + chunk_size
    
    chunk = pickup_points_list[ start:end ]
    chunks_list.append( chunk )
    
    print( ith, start, end, ( end - start ), len( chunk ) )
    
print( len( chunks_list ) )

records 988
chunk_size 98
0 0 98 98 98
1 98 196 98 98
2 196 294 98 98
3 294 392 98 98
4 392 490 98 98
5 490 588 98 98
6 588 686 98 98
7 686 784 98 98
8 784 882 98 98
9 882 988 106 106
10


## Declare Target Function to Run in Parallel Processes

In [36]:
###################################################################################################
# declare where we're dumping results
###################################################################################################
#directory_name = "uber-simulation-10-percent"
directory_name = "uber-simulation"
def process_trips_in_chunk( data_grp, pickup_points_chunk ):
    
    # for calculating ETA
    points_count = len( pickup_points_chunk )

    # get thread name from multiprocessing library
    thread_name = current_process().name
    count = 1
    start_time = get_time()
    
    # search parameters
    rounding_places = 2
    bin_width = 0.01
    search_margin_degs = 0.01
    max_wait_mins = 10
    max_delta = 90

    results_buffer = []
    errors_buffer = []

    for pickup_point in pickup_points_chunk:

        binned_lat, binned_lon = pickup_point
        
        for day in range( 7 ):
            
            for hour in range( 24 ):
                
                # get random bearing every time
                my_bearing = random.uniform( 0.0, 360.0 )
                # add a bit if randomicity to current lat/lon
                random_lat = random.uniform( binned_lat - bin_width / 2, binned_lat + bin_width / 2 )
                random_lon = random.uniform( binned_lon - bin_width / 2, binned_lon + bin_width / 2 )
                
                # get pared down list of taxis near me, closest to my bearing
                results = pd.DataFrame()
                results = get_candidate_list( day, hour, random_lat, random_lon, my_bearing, rounding_places, search_margin_degs, max_delta, max_wait_mins )
                # can't do "results == None"! http://thomas-cokelaer.info/blog/2014/06/pandas-how-to-compare-dataframe-with-none/
                if results is None:
                    errors_buffer.append( str( day ) + "," + str( hour ) + "," + str( binned_lat ) + "," + str( binned_lon ) + "," + str( my_bearing ) + "," + str( random_lat ) + "," + str( random_lon ) )
                else:
                    results_buffer.append( results )

        seconds = time.time() - start_time
        points_per_second = ( count * 1.0 ) / seconds
        percent_completed = ( ( count * 1.0 ) / points_count ) * 100.0
        points_remaining = points_count - count
        minutes_remaining = ( points_remaining / points_per_second ) / 60.0
        print( "[%s] [%d] of [%d] points, running [%f] minutes @ [%d] points/s, [%f]p/c complete, ETA [%f] minutes" % ( thread_name, count, points_count, seconds / 60, points_per_second, percent_completed, minutes_remaining ) )
        count += 1
        
    # write results
    results_df = pd.concat( results_buffer )
    file_name = "data/" + directory_name + "/points-" + thread_name.lower() + ".csv"
    results_df.to_csv( file_name, index=False )
    print( "[%s] DONE! Wrote [%d] points to [%s]" % ( thread_name, count - 1, file_name ) )
     
    # write errors: file
    file_name = "data/" + directory_name + "/points-" + thread_name.lower() + "-errors.csv"
    with open( file_name, 'w' ) as file:
        for line in errors_buffer:
            file.write( line )
            file.write( '\n' )
    

## Start n Separate Processes
Probably 10 cores, leaving 2 cores free so I can do other things while this chews its way through data.

Previous full run on 2018.05.07 @ 12:32pm
Time to process all chunks: [24.953591577212016] minutes

In [37]:
# from https://www.blog.pythonlibrary.org/2016/08/02/python-201-a-multiprocessing-tutorial/
# THIS IS MULTI-PROCESS, *NOT* MULTITHREADED! Uses 100% of 10 cores 
start_time = get_time()    
procs = []

for ith in range( 0, chunks ):
    
    proc = Process( target=process_trips_in_chunk, args=( trips, chunks_list[ ith ] ) )
    procs.append( proc )
    proc.start()

for proc in procs:
    proc.join()
    print( "Process on this core: Done!" )
    
end_time = get_time()
print ( "Time to process all chunks: [%s] minutes" % ( str( ( end_time - start_time ) / 60 ) ) )    

2018.05.09 13:42
2018.05.09 13:42
2018.05.09 13:42
2018.05.09 13:42
2018.05.09 13:42
2018.05.09 13:42
2018.05.09 13:42
2018.05.09 13:42
2018.05.09 13:42
2018.05.09 13:42
2018.05.09 13:42
[Process-39] [1] of [98] points, running [0.079395] minutes @ [0] points/s, [1.020408]p/c complete, ETA [7.701349] minutes
[Process-35] [1] of [98] points, running [0.099739] minutes @ [0] points/s, [1.020408]p/c complete, ETA [9.674713] minutes
[Process-33] [1] of [98] points, running [0.105934] minutes @ [0] points/s, [1.020408]p/c complete, ETA [10.275627] minutes
[Process-31] [1] of [98] points, running [0.109611] minutes @ [0] points/s, [1.020408]p/c complete, ETA [10.632280] minutes
[Process-34] [1] of [98] points, running [0.117594] minutes @ [0] points/s, [1.020408]p/c complete, ETA [11.406607] minutes
[Process-32] [1] of [98] points, running [0.122543] minutes @ [0] points/s, [1.020408]p/c complete, ETA [11.886632] minutes
[Process-40] [1] of [106] points, running [0.151298] minutes @ [0] poin

[Process-35] [6] of [98] points, running [1.474302] minutes @ [0] points/s, [6.122449]p/c complete, ETA [22.605968] minutes
[Process-37] [10] of [98] points, running [1.482491] minutes @ [0] points/s, [10.204082]p/c complete, ETA [13.045920] minutes
[Process-38] [7] of [98] points, running [1.502877] minutes @ [0] points/s, [7.142857]p/c complete, ETA [19.537405] minutes
[Process-39] [10] of [98] points, running [1.513926] minutes @ [0] points/s, [10.204082]p/c complete, ETA [13.322553] minutes
[Process-37] [11] of [98] points, running [1.528476] minutes @ [0] points/s, [11.224490]p/c complete, ETA [12.088856] minutes
[Process-39] [11] of [98] points, running [1.567317] minutes @ [0] points/s, [11.224490]p/c complete, ETA [12.396049] minutes
[Process-38] [8] of [98] points, running [1.592990] minutes @ [0] points/s, [8.163265]p/c complete, ETA [17.921140] minutes
[Process-37] [12] of [98] points, running [1.603997] minutes @ [0] points/s, [12.244898]p/c complete, ETA [11.495312] minute

[Process-37] [21] of [98] points, running [3.181235] minutes @ [0] points/s, [21.428571]p/c complete, ETA [11.664527] minutes
[Process-34] [14] of [98] points, running [3.200410] minutes @ [0] points/s, [14.285714]p/c complete, ETA [19.202458] minutes
[Process-40] [9] of [106] points, running [3.223108] minutes @ [0] points/s, [8.490566]p/c complete, ETA [34.737947] minutes
[Process-35] [10] of [98] points, running [3.248769] minutes @ [0] points/s, [10.204082]p/c complete, ETA [28.589169] minutes
[Process-39] [17] of [98] points, running [3.249650] minutes @ [0] points/s, [17.346939]p/c complete, ETA [15.483627] minutes
[Process-37] [22] of [98] points, running [3.283495] minutes @ [0] points/s, [22.448980]p/c complete, ETA [11.342981] minutes
[Process-31] [12] of [98] points, running [3.286621] minutes @ [0] points/s, [12.244898]p/c complete, ETA [23.554116] minutes
[Process-40] [10] of [106] points, running [3.297353] minutes @ [0] points/s, [9.433962]p/c complete, ETA [31.654585] m

[Process-33] [15] of [98] points, running [4.756242] minutes @ [0] points/s, [15.306122]p/c complete, ETA [26.317872] minutes
[Process-34] [20] of [98] points, running [4.775652] minutes @ [0] points/s, [20.408163]p/c complete, ETA [18.625044] minutes
[Process-33] [16] of [98] points, running [4.802047] minutes @ [0] points/s, [16.326531]p/c complete, ETA [24.610492] minutes
[Process-34] [21] of [98] points, running [4.835471] minutes @ [0] points/s, [21.428571]p/c complete, ETA [17.730059] minutes
[Process-34] [22] of [98] points, running [4.890254] minutes @ [0] points/s, [22.448980]p/c complete, ETA [16.893606] minutes
[Process-35] [25] of [98] points, running [4.894170] minutes @ [0] points/s, [25.510204]p/c complete, ETA [14.290976] minutes
[Process-35] [26] of [98] points, running [4.944022] minutes @ [0] points/s, [26.530612]p/c complete, ETA [13.691139] minutes
[Process-35] [27] of [98] points, running [5.090042] minutes @ [0] points/s, [27.551020]p/c complete, ETA [13.384925] 

[Process-40] [21] of [106] points, running [6.695388] minutes @ [0] points/s, [19.811321]p/c complete, ETA [27.100381] minutes
[Process-37] [39] of [98] points, running [6.702110] minutes @ [0] points/s, [39.795918]p/c complete, ETA [10.139090] minutes
[Process-37] [40] of [98] points, running [6.740843] minutes @ [0] points/s, [40.816327]p/c complete, ETA [9.774223] minutes
[Process-31] [24] of [98] points, running [6.792676] minutes @ [0] points/s, [24.489796]p/c complete, ETA [20.944085] minutes
[Process-37] [41] of [98] points, running [6.880613] minutes @ [0] points/s, [41.836735]p/c complete, ETA [9.565730] minutes
[Process-31] [25] of [98] points, running [6.886101] minutes @ [0] points/s, [25.510204]p/c complete, ETA [20.107416] minutes
[Process-33] [24] of [98] points, running [6.908246] minutes @ [0] points/s, [24.489796]p/c complete, ETA [21.300425] minutes
[Process-38] [28] of [98] points, running [6.916265] minutes @ [0] points/s, [28.571429]p/c complete, ETA [17.290663] m

...

[Process-34] [69] of [98] points, running [16.127013] minutes @ [0] points/s, [70.408163]p/c complete, ETA [6.778020] minutes
[Process-38] [80] of [98] points, running [16.127033] minutes @ [0] points/s, [81.632653]p/c complete, ETA [3.628583] minutes
[Process-36] [83] of [98] points, running [16.147789] minutes @ [0] points/s, [84.693878]p/c complete, ETA [2.918275] minutes
[Process-35] [85] of [98] points, running [16.171182] minutes @ [0] points/s, [86.734694]p/c complete, ETA [2.473240] minutes
[Process-32] [64] of [98] points, running [16.180403] minutes @ [0] points/s, [65.306122]p/c complete, ETA [8.595839] minutes
[Process-36] [84] of [98] points, running [16.208789] minutes @ [0] points/s, [85.714286]p/c complete, ETA [2.701465] minutes
[Process-31] [69] of [98] points, running [16.219670] minutes @ [0] points/s, [70.408163]p/c complete, ETA [6.816963] minutes
[Process-38] [81] of [98] points, running [16.234925] minutes @ [0] points/s, [82.653061]p/c complete, ETA [3.407330] 

[Process-39] [71] of [98] points, running [17.487433] minutes @ [0] points/s, [72.448980]p/c complete, ETA [6.650151] minutes
[Process-36] [96] of [98] points, running [17.514524] minutes @ [0] points/s, [97.959184]p/c complete, ETA [0.364886] minutes
[Process-39] [72] of [98] points, running [17.535875] minutes @ [0] points/s, [73.469388]p/c complete, ETA [6.332399] minutes
[Process-36] [97] of [98] points, running [17.568055] minutes @ [0] points/s, [98.979592]p/c complete, ETA [0.181114] minutes
[Process-39] [73] of [98] points, running [17.609760] minutes @ [0] points/s, [74.489796]p/c complete, ETA [6.030740] minutes
[Process-36] [98] of [98] points, running [17.617258] minutes @ [0] points/s, [100.000000]p/c complete, ETA [0.000000] minutes
[Process-39] [74] of [98] points, running [17.675060] minutes @ [0] points/s, [75.510204]p/c complete, ETA [5.732452] minutes
[Process-39] [75] of [98] points, running [17.711299] minutes @ [0] points/s, [76.530612]p/c complete, ETA [5.431465]

[Process-37] [93] of [98] points, running [19.322450] minutes @ [0] points/s, [94.897959]p/c complete, ETA [1.038841] minutes
[Process-32] [80] of [98] points, running [19.374370] minutes @ [0] points/s, [81.632653]p/c complete, ETA [4.359233] minutes
[Process-34] [85] of [98] points, running [19.383495] minutes @ [0] points/s, [86.734694]p/c complete, ETA [2.964535] minutes
[Process-40] [80] of [106] points, running [19.423391] minutes @ [0] points/s, [75.471698]p/c complete, ETA [6.312602] minutes
[Process-34] [86] of [98] points, running [19.436042] minutes @ [0] points/s, [87.755102]p/c complete, ETA [2.712006] minutes
[Process-40] [81] of [106] points, running [19.469634] minutes @ [0] points/s, [76.415094]p/c complete, ETA [6.009146] minutes
[Process-33] [71] of [98] points, running [19.494952] minutes @ [0] points/s, [72.448980]p/c complete, ETA [7.413573] minutes
[Process-35] [95] of [98] points, running [19.538033] minutes @ [0] points/s, [96.938776]p/c complete, ETA [0.616991

[Process-31] [85] of [98] points, running [20.826695] minutes @ [0] points/s, [86.734694]p/c complete, ETA [3.185259] minutes
[Process-40] [96] of [106] points, running [20.846132] minutes @ [0] points/s, [90.566038]p/c complete, ETA [2.171472] minutes
[Process-39] [86] of [98] points, running [20.877792] minutes @ [0] points/s, [87.755102]p/c complete, ETA [2.913180] minutes
[Process-37] [98] of [98] points, running [20.892843] minutes @ [0] points/s, [100.000000]p/c complete, ETA [0.000000] minutes
[Process-40] [97] of [106] points, running [20.898164] minutes @ [0] points/s, [91.509434]p/c complete, ETA [1.939005] minutes
[Process-32] [85] of [98] points, running [20.907084] minutes @ [0] points/s, [86.734694]p/c complete, ETA [3.197554] minutes
[Process-40] [98] of [106] points, running [20.940950] minutes @ [0] points/s, [92.452830]p/c complete, ETA [1.709465] minutes
[Process-32] [86] of [98] points, running [20.942093] minutes @ [0] points/s, [87.755102]p/c complete, ETA [2.9221

[Process-39] [97] of [98] points, running [23.282974] minutes @ [0] points/s, [98.979592]p/c complete, ETA [0.240031] minutes
[Process-31] [97] of [98] points, running [23.286091] minutes @ [0] points/s, [98.979592]p/c complete, ETA [0.240063] minutes
[Process-31] [98] of [98] points, running [23.341281] minutes @ [0] points/s, [100.000000]p/c complete, ETA [0.000000] minutes
[Process-39] [98] of [98] points, running [23.368492] minutes @ [0] points/s, [100.000000]p/c complete, ETA [0.000000] minutes
[Process-31] DONE! Wrote [98] points to [data/uber-simulation/points-process-31.csv]
Process on this core: Done!
Process on this core: Done!
[Process-39] DONE! Wrote [98] points to [data/uber-simulation/points-process-39.csv]
[Process-33] [91] of [98] points, running [23.773495] minutes @ [0] points/s, [92.857143]p/c complete, ETA [1.828730] minutes
[Process-33] [92] of [98] points, running [23.796578] minutes @ [0] points/s, [93.877551]p/c complete, ETA [1.551951] minutes
[Process-33] [93